In [1]:
import igraph as ig
import pathlib
import pickle
from tqdm.notebook import tqdm, trange
import time
import networkx as nx
import pandas as pd
import numpy as np

Specification of data and supplementary dirs

In [2]:
# data_type = '2009-2010'
data_type = '2000-2001'
# data_type = '1999-2000'
# data_type = 'subsample'
DATA_PATH = f'../data arxiv/hyperedges_list_arxiv_{data_type}.txt'
LINEGRAPH_EDGES_PATH = f'../suplementary/linegraphs/linegraph_weighted_edges_list_{data_type}.txt'
LINEGRAPH_NODES_PATH = f'../suplementary/linegraphs/linegraph_weighted_nodes_list_{data_type}.txt'
DIST_DIR = "../suplementary/dist"
CLIQUE_PATH = f"../suplementary/clique-projections/{data_type}-proj-graph.txt"

### Data reading

In [3]:
data_path = DATA_PATH
with open(data_path) as file:
    line = file.readline()

if data_type in ["subsample", "test"]:
    hyperedges_list = line.split('], ')
    hyperedges_list = [x.replace('[', '') for x in hyperedges_list]
    hyperedges_list = [x.replace(']', '') for x in hyperedges_list]  # removes the last "]""
else:
    hyperedges_list = line.split(']", ')
    hyperedges_list = [x.replace('"[', '') for x in hyperedges_list]
    hyperedges_list = [x.replace(']"', '') for x in hyperedges_list]  # removes the last "]""

print(len(hyperedges_list))
print(hyperedges_list[0])
print(hyperedges_list[-1])

8914
'hep-ex', 'physics.comp-ph'
'math.ag', 'math.cv'


In [4]:
hyperedges_list, counts_list = np.unique(hyperedges_list,
                                         return_counts=True)  # counts_list contains info about frequencies of each hyperedge
print(len(hyperedges_list))
print(len(counts_list))

hyperedges_list = [x.replace("'", '') for x in hyperedges_list]
hyperedges_list = [x.split(', ') for x in hyperedges_list]
print(hyperedges_list[:3])
print(counts_list[:3])

2205
2205
[['astro-ph', 'astro-ph.ga'], ['astro-ph', 'cond-mat'], ['astro-ph', 'cond-mat', 'gr-qc', 'hep-ph']]
[1 1 2]


In [5]:
# left hyperedges of size 1
hyperedges_list_ids = [i for i in range(len(hyperedges_list)) if len(hyperedges_list[i]) > 1]
print(len(hyperedges_list_ids))
if len(hyperedges_list_ids) < len(hyperedges_list):
    hyperedges_list = [hyperedges_list[i] for i in hyperedges_list_ids]
    counts_list = [counts_list[i] for i in hyperedges_list_ids]

2205


In [6]:
# create supplementary dictionary containing nodes as keys and hyperedges to which they are incident as values. It helps to access needed hyperedges faster
def get_nodes_to_edges(edges_to_nodes_dict):
    nodes_to_edges_dict = {}
    for edge, nodes in edges_to_nodes_dict.items():
        for node in nodes:
            if node in nodes_to_edges_dict.keys():
                nodes_to_edges_dict[node].append(edge)
            else:
                nodes_to_edges_dict[node] = [edge]
    return nodes_to_edges_dict

In [7]:
edges_to_nodes_dict = {i: hyperedges_list[i] for i in
                       range(len(hyperedges_list))}  # key = hyperedge, value = nodes in hyperedge
edges_to_counts_dict = {i: counts_list[i] for i in range(len(counts_list))}
nodes_to_edges_dict = get_nodes_to_edges(
    edges_to_nodes_dict)  # key = node, value = list of hyperedges in which the node participates
print(f'nodes count = {len(nodes_to_edges_dict.keys())}')

nodes count = 135


### Line-graph construction functions

In [8]:
# weight_function is needed for creation of weighted line graph, w1 (w2) is e1's (e2's) weight in weighted hypergraph
def weight_function(e1, e2, w1, w2):
    if e1 == e2:
        return (1 / w1) * ((1 / 3) * (len(e1) + 1) - 1)
    union_len = len(set(e1).union(e2))
    intersect_len = len(set(e1).intersection(e2))
    return (1 / 2) * (1 / w1 + 1 / w2) * ((1 / 3) * union_len * (1 + 1 / intersect_len) - 1)


def generate_weighted_line_graph(edges_to_nodes_dict, edges_to_counts_dict) -> ig.Graph:
    file_path = pathlib.Path(LINEGRAPH_EDGES_PATH)

    if not file_path.exists():

        items = list(edges_to_nodes_dict.items())
        line_nodes_ids_dict = {items[i][0]: i for i in
                               range(len(items))}  # in line-graph nodes = hyperedges in intial hypergraph
        line_edges_list = []
        line_edges_weights_list = []

        for i in range(len(items)):
            edge, nodes = items[i]
            line_edges_list.append([i, i])  # our line-graph has self-loops
            line_edges_weights_list.append(weight_function(nodes, nodes,
                                                           edges_to_counts_dict[edge],
                                                           edges_to_counts_dict[edge]))  # add weights of self-loops

        for i in tqdm(range(len(items))):
            edge, nodes = items[i]
            set_nodes = set(nodes)
            neighbors = [k for k, v in items if
                         not k == edge and
                         (len(v) >= len(nodes)) and
                         not set_nodes.isdisjoint(v)]  # find hyperedges that intersect with the current one

            if len(neighbors) > 0:
                for neighbor in neighbors:
                    line_edges_list.append([line_nodes_ids_dict[edge], line_nodes_ids_dict[
                        neighbor]])  # each intersection leads to edge in the line-graph
                    line_edges_weights_list.append(weight_function(nodes, edges_to_nodes_dict[neighbor],
                                                                   edges_to_counts_dict[edge],
                                                                   edges_to_counts_dict[neighbor]))

        G = ig.Graph(n=len(items), edges=line_edges_list,
                     edge_attrs={'weight': line_edges_weights_list},
                     vertex_attrs={'label': list(line_nodes_ids_dict.keys())})
        edges_list = G.get_edgelist()
        print('saving line graph')
        with open(LINEGRAPH_EDGES_PATH, 'w') as file:
            lines = []
            for edge_id in range(len(edges_list)):
                edge_weight = G.es[edge_id]["weight"]
                lines.append(f'{edges_list[edge_id][0]} {edges_list[edge_id][1]} {edge_weight}\n')
            file.writelines(lines)
        with open(LINEGRAPH_NODES_PATH, 'w') as file:
            file.writelines([str(G.vs[i]['label']) + '\n' for i in range(len(items))])

        print('line graph saved')
        del G

    with open(LINEGRAPH_EDGES_PATH, 'r') as file:
        lines = file.readlines()
    with open(LINEGRAPH_NODES_PATH, 'r') as file:
        lines_nodes = file.readlines()
    edges_list = []
    weights = []
    nodes = []
    print('preparing data for line graph creation')
    print(len(lines))
    for line in lines:
        from_node, to_node, weight = line.split(' ')
        from_node = int(from_node)
        to_node = int(to_node)
        if from_node not in nodes:
            nodes.append(from_node)
        if to_node not in nodes:
            nodes.append(to_node)
        weight = float(weight)
        edges_list.append((from_node, to_node))
        weights.append(weight)

    print('line graph creation')
    G = ig.Graph(n=len(nodes), edges=edges_list,
                 edge_attrs={'weight': weights},
                 vertex_attrs={'label': [int(x) for x in lines_nodes]})
    return G



### Hypergraph distance calculation function

In [9]:
# to find shortest path from node u to node v one should, first, find path in line-graph from all nodes, representing hyperedges to which u belongs, to all nodes, representing all hyperedges to which v belongs 
# each hyperedge contains sereis of nodes, therefore,  paths between nodes in the line-graph might be the same for different node pairs.
# To fasten calculation we can save shortest paths that we have already found  
# global_all_path_dict contains information about previously found paths between nodes in line-graph (representing hyperedges in hypergraph)

def get_distance_new(nodes_to_edges_dict, edges_to_counts_dict, line_graph : ig.Graph, u, v, global_all_path_dict={},
                     return_path=False):
    from_edges = nodes_to_edges_dict[u]  # from_edges - edges incident to node u
    to_edges = nodes_to_edges_dict[v]  # to_edges - edges incident to node v 
    if len(from_edges) < len(to_edges):
        u_edges = from_edges
        v_edges = to_edges
    else:
        u_edges = to_edges
        v_edges = from_edges

    line_nodes_ids_dict = {line_graph.vs["label"][i]: i for i in range(len(line_graph.vs["label"]))}
    line_ids_nodes_dict = {i: line_graph.vs["label"][i] for i in range(len(line_graph.vs["label"]))}

    uv_dist = []
    uv_paths = []
    
    for u_e in u_edges:
        if u_e in global_all_path_dict.keys():
            all_paths_dict = global_all_path_dict[u_e].copy()
        else:
            all_paths_dict = {}
        to_linegraph_nodes = [v_e for v_e in v_edges if
                              v_e not in all_paths_dict.keys() and not v_e == u_e]  # nodes in line-graph to which paths from u_e have not been found previously 

        drop_ve = []
        for v_e in to_linegraph_nodes:
            if v_e in global_all_path_dict.keys():  # paths are not directed. If we already have path from v_e to u_e it is the same path as from u_e to v_e and we should not find it once more
                if u_e in global_all_path_dict[v_e].keys():
                    all_paths_dict[v_e] = [list(np.flip(x)) for x in global_all_path_dict[v_e][u_e]]
                    drop_ve.append(v_e)

        all_paths = line_graph.get_all_shortest_paths(line_nodes_ids_dict[u_e],
                                                      to=[line_nodes_ids_dict[x] for x in to_linegraph_nodes if
                                                          x not in drop_ve],
                                                      weights=line_graph.es[
                                                          "weight"])  # calculation of absent shortest paths 

        for i in range(len(all_paths)):  # collecting of the new shortest paths
            if len(all_paths[i]) > 1:
                if not line_ids_nodes_dict[all_paths[i][-1]] in all_paths_dict.keys():
                    all_paths_dict[line_ids_nodes_dict[all_paths[i][-1]]] = [all_paths[i]]

                    if not line_ids_nodes_dict[all_paths[i][-1]] in global_all_path_dict.keys():
                        global_all_path_dict[line_ids_nodes_dict[all_paths[i][-1]]] = {
                            u_e: [list(np.flip(all_paths[i]))]}
                    else:
                        global_all_path_dict[line_ids_nodes_dict[all_paths[i][-1]]][u_e] = [list(np.flip(all_paths[i]))]
                else:
                    all_paths_dict[line_ids_nodes_dict[all_paths[i][-1]]].append(all_paths[i])
                    global_all_path_dict[line_ids_nodes_dict[all_paths[i][-1]]][u_e].append(list(np.flip(all_paths[i])))

        global_all_path_dict[
            u_e] = all_paths_dict.copy()  # saving of the renewed collection of the shortest paths from u_e

        for v_e in v_edges:  # calculate distances between u and v corresponding to every u_e -> v_e 
            if u_e == v_e:
                dist = line_graph.es["weight"][line_graph.get_eid(line_nodes_ids_dict[u_e], line_nodes_ids_dict[v_e])]
                uv_dist.append(dist + 1 / edges_to_counts_dict[u_e])  # modification for weighted hypergraph
                uv_paths.append([line_graph.get_eid(line_nodes_ids_dict[u_e], line_nodes_ids_dict[v_e])])
            else:
                paths = all_paths_dict[v_e]
                uv_paths.append(paths)
                if len(paths[0]) > 0:
                    distance = 0
                for i in range(len(paths[0]) - 1):
                    start = paths[0][i]
                    end = paths[0][i + 1]
                    e = line_graph.get_eid(start, end)
                    distance += line_graph.es[e]["weight"]
                    # uv_paths[-1].append((start,end))
                distance += (1 / 2) * (1 / edges_to_counts_dict[u_e] + 1 / edges_to_counts_dict[v_e])
                uv_dist.append(distance)

    if not return_path:
        return min(uv_dist), global_all_path_dict

    min_indexes = [i for i in range(len(uv_dist)) if uv_dist[i] == min(
        uv_dist)]  # indexes corresponding to minimal distance between differen u_e and v_e pairs

    paths = []
    for i in min_indexes:
        if len(uv_paths[i]) > 1:
            paths_str = [",".join([str(y) for y in x]) for x in uv_paths[i]]
            paths_str = np.unique(paths_str)
            paths_tmp = [x.split(',') for x in paths_str]
            paths_tmp = [[int(y) for y in x] for x in paths_tmp]
            paths.extend(paths_tmp)
        else:
            paths.extend(uv_paths[i].copy())

    return min(uv_dist), paths, global_all_path_dict

In [10]:
import queue
from heapq import *
# пока требует доработок
def get_distance_new_alg_v2(
        nodes_from,
        nodes_to,
        edges_to_counts_dict,
        line_graph : nx.Graph):
    from_edges = set(nodes_from)  # from_edges - edges incident to node u
    to_edges = set(nodes_to)  # to_edges - edges incident to node v 
    loops = list(from_edges.intersection(to_edges))
    if len(loops) > 0:
        loops_w = [line_graph[u][u]['weight'] + 1/edges_to_counts_dict[u] for u in loops]
        min_loop = np.argmin(loops_w)
        min_node_loop = loops[min_loop]
        min_loop = loops_w[min_loop]
    else:
        min_loop = 100_000
        min_node_loop = None
                    
    uv_dist = {}
    uv_path = {}
    for u in line_graph.nodes():
        uv_dist[u] = float('inf')
        uv_path[u] = None
    
    pq = queue.PriorityQueue()
    for u in from_edges:
        for v in line_graph[u]:
            if v == u:
                continue
            prev_val = uv_dist[v]
            new_val = line_graph[u][v]['weight'] + 1/2/edges_to_counts_dict[u]
            if new_val < prev_val:
                uv_dist[v] = new_val
                uv_path[v] = u
    for u in uv_dist:
        if not np.isinf(uv_dist[u]): 
            pq.put((uv_dist[u], u))
    min_node = None
    min_val = float('inf')
    while not pq.empty():
        u = pq.get()
        # if u[0] > min_val + 0.5:
        #     break
        for v in line_graph[u[1]]:
            if v == u[1]:
                continue
            new_w = u[0] + line_graph[u[1]][v]['weight']
            if uv_dist[v] > new_w:
                uv_dist[v] = new_w
                uv_path[v] = u[1]
                pq.put((new_w, v))
                if v in to_edges:
                    w = new_w + 1/2/edges_to_counts_dict[v]
                    if w < min_val:
                        min_val = w
                        min_node = v
    
    if min_loop < min_val:
        return (min_loop, [min_node_loop])
    else:
        path = [min_node]
        p = min_node
        visited = set(path)
        while uv_path[p] is not None:
            p = uv_path[p]
            if p in from_edges:
                path.append(p)    
                break
            if p in visited:
                break
            visited.add(p)
            path.append(p)
        return (min_val, path[::-1])

In [42]:
import queue
from heapq import *
# алгоритм для поиска расстояния между двумя вершинами
# алгоритм состоит из двух частей: учет циклов e->e у учет путей, состоящих более чем из двух вершин
def get_distance_new_alg(
        nodes_from,
        nodes_to,
        edges_to_counts_dict,
        line_graph : nx.Graph):
    from_edges = set(nodes_from)  # from_edges - edges incident to node u
    to_edges = set(nodes_to)  # to_edges - edges incident to node v 
    # циклы вида e->e
    loops = list(from_edges.intersection(to_edges))
    
    if len(loops) > 0:
        # ищем минимальный цикл и минимальную ноду
        loops_w = [line_graph[u][u]['weight'] + 1/edges_to_counts_dict[u] for u in loops]
        min_loop = np.argmin(loops_w)
        min_node_loop = loops[min_loop]
        min_loop = loops_w[min_loop]
    else:
        min_loop = 100_000
        min_node_loop = None
    # тут хранится расстояние до вершины, и вершина из которой быстрее всего прийти в данную вершину
    uv_dist = {}
    uv_path = {}
    for u in line_graph.nodes():
        uv_dist[u] = float('inf')
        uv_path[u] = None
    
    pq = []
    # сразу посещаем все вершины, до которых можно дойти из стартовых
    # это нужно чтобы ... todo (описать случай)
    for u in from_edges:
        for v in line_graph[u]:
            if v == u:
                continue
            prev_val = uv_dist[v]
            new_val = line_graph[u][v]['weight'] + 1/2/edges_to_counts_dict[u]
            if new_val < prev_val:
                uv_dist[v] = new_val
                uv_path[v] = u
    
    for u in uv_dist:
        if not np.isinf(uv_dist[u]): 
            heappush(pq,(uv_dist[u], u))
            # pq.put((uv_dist[u], u))
    
    # основной алгоритм
    while len(pq) > 0:
        u = heappop(pq)#pq.get()
        for v in line_graph[u[1]]:
            if v == u[1]:
                continue
            new_w = u[0] + line_graph[u[1]][v]['weight']
            if uv_dist[v] > new_w:
                uv_dist[v] = new_w
                uv_path[v] = u[1]
                heappush(pq,(new_w, v))
                # pq.put((new_w, v))
    # выбираем мин путь
    min_node = None
    min_val = float('inf')
    for v in to_edges:
        uv_dist[v]+=1/2/edges_to_counts_dict[v]
        if min_val > uv_dist[v]:
            min_val = uv_dist[v]
            min_node = v
    if min_loop < min_val:
        return (min_loop, [min_node_loop])
    else:
        path = [min_node]
        p = min_node
        visited = set(path)
        
        # востановление пути todo проверить и допистаь 
        # while uv_path[p] is not None:
        #     p = uv_path[p]
        #     if p in from_edges:
        #         path.append(p)    
        #         break
        #     if p in visited:
        #         break
        #     visited.add(p)
        #     path.append(p)
        return (min_val, path[::-1])

In [12]:
# suplementary function to calculate particulare path length
def calc_path_weight(edges_to_counts_dict, line_graph, path):
    line_ids_nodes_dict = {i: line_graph.vs["label"][i] for i in range(len(line_graph.vs["label"]))}
    length = 0
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        e = line_graph.get_eid(start, end)
        length += line_graph.es[e]["weight"]
    length += (1 / 2) * (1 / edges_to_counts_dict[line_ids_nodes_dict[path[0]]] + \
                         1 / edges_to_counts_dict[line_ids_nodes_dict[path[-1]]])
    return length

### Line-graph generation

In [13]:
# if we constructed line-graph once it is saved into suplementary/linegraphs to fasten further calculation
line_graph = generate_weighted_line_graph(edges_to_nodes_dict, edges_to_counts_dict)

preparing data for line graph creation
539052
line graph creation


### Computation of distances between specified pairs of nodes

In [14]:
G = line_graph.to_networkx()
Q = nx.Graph()
for u,du in G.nodes(data=True):
    Q.add_node(u)
for u in G.nodes():
    for v in G[u]:
        data = G[u][v]
        Q.add_edge(u,v,weight = float(data[0]['weight']))
del G
print(len(Q.nodes))
print(len(Q.edges))

2205
431649


In [15]:
import sys
ss = 100**100
sys.getsizeof(ss) / 8 / 1025

0.014146341463414635

In [16]:
# from threading import Semaphore
# # from concurrent.futures.process import ProcessPoolExecutor as Pool
# from multiprocessing import Pool
# import numpy as np
# def calc(data):
#     res = np.cos(np.cos(np.cos(data[0])))
#     return res
# 
# def get_data(semaphore):
#     data = (1, Q)
#     for i in range(11):
#         semaphore.acquire()
#         yield data
# 
# s = Semaphore(10000)
# with Pool(10) as p:
#     for i in range(1000):
#         data = get_data(s)
#         res = []
#         for r in tqdm(p.imap_unordered(calc, data), total=11):
#             res.append(r)
#             s.release()
# 
#         print(len(res))

In [17]:
# from multiprocessing import Pool
# import numpy as np
# def calc(data):
#     return np.cos(np.cos(np.cos(data[0])))
# with Pool(10) as p:
#     for i in range(1000):
#         data = [(j, Q) for j in range(1000_000)]
#         res = p.map(calc, data)
#         print(len(res))

In [34]:
from time import sleep
# from multiprocessing import Pool
from threading import Semaphore
from concurrent.futures.process import ProcessPoolExecutor as Pool
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1
# start_id = 120
# final_id = 120


def do_calc(data):
    nodes_to_edges_dict, edges_to_counts_dict, Q, nodes_from, us, i = data
    result = {}
    # чтобы нормально подгрузился tqdm
    sleep(i/10)
    print('start:', i)
    points_data = [(node_from, node_to) for node_from in nodes_from for node_to in us]
    for node_from, u in tqdm(points_data, position=i):
        if node_from not in result:
            result[node_from] = {}
        # print(u)
        if u == node_from:
            result[node_from][u] = 0
            continue
        data, path = get_distance_new_alg(nodes_to_edges_dict[node_from], nodes_to_edges_dict[u], edges_to_counts_dict,Q)
        # old = old_res['astro-ph'][i]
        # delta = old - data
        result[node_from][u] = data
        # result.append((node_from, u, data))
        # print(my_round(delta), path)
    return result

WORKERS = 10
to_points = list(nodes_to_edges_dict.keys())
# with Pool(WORKERS) as p:
#             
#     for id_from in trange(start_id, final_id + 1,desc='find_paths', position = 1):
#         time_start = time.time()
#         # if id_from == 3:
#         #     break
#         node_from = node_names[id_from]
#         dists = []
#         paths = []
#         print(id_from, node_from)
#         data= [(nodes_to_edges_dict, edges_to_counts_dict, Q, node_from, [u for u in to_points[i::WORKERS]], i) for i in range(WORKERS)]
#         
#         data = list(tqdm(p.imap_unordered(do_calc, data), total=len(data)))
#         for dd in data:
#             for d in dd:
#                 if d[0] not in dists_dict:
#                     dists_dict[d[0]] = {}
#                 dists_dict[d[0]][d[1]] = d[2]
#         with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle',
#                   'wb') as handle:  # we save calculated distances and corresponding shortest paths to fasten furter analysis
#             pickle.dump((dists_dict, paths_dict), handle)
#         time_term = time.time()
#         print(f'Elapsed time {time_term - time_start}')

многопоточная реализация

In [44]:
from multiprocessing import Pool
to_points = list(nodes_to_edges_dict.keys())
WORKERS = 20
# данные для расчета на нескольких потоках 
# каждому потоку дается список вида (nodes_to_edges_dict, edges_to_counts_dict, line_graph, ноды от которых надо искать пути данному потоку, все ноды до которых надо дойти)
data = [(nodes_to_edges_dict, edges_to_counts_dict, Q, [node_names[id_from] for id_from in range(start_id+i, final_id+1,WORKERS)], [u for u in to_points], i) for i in range(WORKERS)]
time_start = time.time()
with Pool(WORKERS) as p:
    # res это список словарей вида {node_from: {node_to: length}}
    res = p.map(do_calc, data)
time_term = time.time()
new_res = {}
print(res)
for r in res:
    for f in r:
        if f not in new_res:
            new_res[f] = {}
        for t in r[f]:
            new_res[f][t] = r[f][t]
with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle',
                  'wb') as handle:  # we save calculated distances and corresponding shortest paths to fasten furter analysis
            pickle.dump(res, handle)

start: 0


  0%|          | 0/945 [00:00<?, ?it/s]

start: 1


  0%|          | 0/945 [00:00<?, ?it/s]

start: 2


...

start: 18


  0%|          | 0/810 [00:00<?, ?it/s]

start: 19


  0%|          | 0/810 [00:00<?, ?it/s]

[{'astro-ph': {'astro-ph': 0, 'astro-ph.ga': np.float64(1.0), 'cond-mat': np.float64(0.05553181386514719), 'gr-qc': np.float64(0.00980392156862745), 'hep-ph': np.float64(0.007692307692307693), 'nucl-th': np.float64(0.012955465587044534), 'hep-ex': np.float64(0.013406593406593406), 'nucl-ex': np.float64(0.03436694884063305), 'physics.ins-det': np.float64(0.43007326007326), 'physics.atom-ph': np.float64(0.1326923076923077), 'physics.plasm-ph': np.float64(0.125), 'cond-mat.soft': np.float64(0.12169334288899507), 'cond-mat.stat-mech': np.float64(0.11276477146042363), 'nlin.ao': np.float64(0.13245489078822412), 'cond-mat.supr-con': np.float64(0.11707552540885875), 'cs.db': np.float64(0.8333333333333333), 'cs.dl': np.float64(0.8333333333333333), 'cs.ir': np.float64(0.8333333333333333), 'cs.oh': np.float64(1.0494695428028762), 'physics.data-an': np.float64(0.19947802197802197), 'physics.space-ph': np.float64(0.25980392156862747), 'hep-th': np.float64(0.013865147198480532), 'quant-ph': np.floa

In [46]:
print(f'Elapsed time {time_term - time_start}')

Elapsed time 2726.9601509571075


In [33]:
# new_res

{'astro-ph': {'astro-ph': 0, 'cs.cy': np.float64(1.6666666666666665)},
 'cs.cy': {'astro-ph': np.float64(1.6666666666666665), 'cs.cy': 0}}

In [ ]:
# from multiprocessing import Pool
# to_points = list(nodes_to_edges_dict.keys())
# def get_data():
#     for id_from in range(start_id, final_id + 1):
#         node_from = node_names[id_from]
#         for u in to_points:
#             print(f'init {u}')
#             yield (nodes_to_edges_dict, edges_to_counts_dict, Q, node_from, [u])
# with Pool(WORKERS) as p:
#     for r in p.imap_unordered(do_calc, get_data()):
#         print(r)

In [48]:
dists_dict = new_res
dists_dict = new_res


In [61]:

for p1 in dists_dict:
    for p2 in dists_dict:
        for p3 in dists_dict:
            d13 = dists_dict[p1][p3]
            d12 = dists_dict[p1][p2]
            d23 = dists_dict[p2][p3]
            if d13 > d12 + d23:
                print(d13,d12,d23)

0.8871984805318137 0.09199786399165054 0.6666666666666666
0.9149762583095916 0.09199786399165054 0.6666666666666666
0.9149762583095916 0.09199786399165054 0.6666666666666666
1.9983095916429248 0.09199786399165054 1.7777777777777777
1.5538651471984806 0.09199786399165054 1.3333333333333333
0.9496984805318138 0.09199786399165054 0.8125
0.5538651471984805 0.09199786399165054 0.3333333333333333
1.0538651471984803 0.09199786399165054 0.8333333333333334
1.9983095916429248 0.09199786399165054 1.7777777777777777
1.0538651471984806 0.09199786399165054 0.8333333333333333
0.8871984805318137 0.09199786399165054 0.6666666666666666
1.0737552570885904 0.9856600189933522 0.07142857142857142
1.8058981142314474 0.9856600189933522 0.8035714285714286
1.5737552570885902 0.9856600189933522 0.5714285714285714
0.09199786399165054 0.013865147198480532 0.06565843695777906
0.8025241797811242 0.013865147198480532 0.7761847527472527
0.8871984805318137 0.013865147198480532 0.8609800743846795
0.5538978061547506 0.01

In [49]:
to_points = list(nodes_to_edges_dict.keys())
new_res = {k:[dists_dict[k][i] for i in to_points] for k in dists_dict}
new_res.keys()

dict_keys(['astro-ph', 'physics.space-ph', 'cond-mat.other', 'math.pr', 'cs.ai', 'cs.cy', 'math.gm', 'astro-ph.ga', 'hep-th', 'cond-mat.quant-gas', 'nlin.cg', 'cs.dm', 'cs.si', 'math.st', 'cond-mat', 'quant-ph', 'cs.lg', 'q-bio.bm', 'cs.it', 'cs.mm', 'stat.th', 'gr-qc', 'math.ds', 'hep-lat', 'q-bio.nc', 'math.it', 'cs.ms', 'math.ct', 'hep-ph', 'math-ph', 'nlin.si', 'q-fin.tr', 'cs.cl', 'cs.sd', 'math.gr', 'nucl-th', 'math.mp', 'math.ap', 'q-fin.st', 'cs.lo', 'cs.os', 'math.at', 'hep-ex', 'nlin.ps', 'math.cv', 'math.qa', 'cs.ma', 'cs.se', 'math.ac', 'nucl-ex', 'physics.ao-ph', 'math.fa', 'q-fin.rm', 'cs.na', 'cs.sc', 'math.gn', 'physics.ins-det', 'physics.flu-dyn', 'physics.optics', 'cs.ce', 'cs.pl', 'cs.ro', 'math.ho', 'physics.atom-ph', 'physics.comp-ph', 'q-bio.cb', 'q-fin.pr', 'cs.ar', 'math.sp', 'q-bio.gn', 'physics.plasm-ph', 'physics.hist-ph', 'physics.ed-ph', 'math.nt', 'cs.ds', 'math.sg', 'q-bio.ot', 'cond-mat.soft', 'physics.pop-ph', 'physics.soc-ph', 'q-fin.gn', 'cs.cv', 'mat

In [ ]:
idx=[40, 897, 999]
for i in idx:
    name = ''
    for x in edges_to_nodes_dict[i]:
        name+=x + ' '
    print(f'{name} w = {edges_to_counts_dict[i]}')

In [ ]:
# -0.003700000000000002 0.053897806154750594 astro-ph math-ph
# -0.003700000000000002 0.053897806154750594 astro-ph math.mp

In [50]:
with open(f'../data/dists_2000-2001_weighted-0-134.pickle', 'rb') as handle:
    old_res, paths = pickle.load(handle)

In [ ]:
old_res['astro-ph']

In [58]:
def my_round(x):
    r = 10000
    return int(r*x)/r

In [59]:
for f in new_res:
    o = old_res[f]
    n = new_res[f]
    for i in range(len(o)):
        delta = my_round(o[i]- n[i])
        if abs(my_round(delta))!=0:
            print(f, new_res[f][i], delta)

In [ ]:
s = 0
e = 0
for i in range(len(old_res['astro-ph'])):
    delta = my_round(old_res['astro-ph'][i]) - my_round(dists_dict['astro-ph'][i])
    if abs(delta)!=0:
        print(old_res['astro-ph'][i], dists_dict['astro-ph'][i],delta/old_res['astro-ph'][i] * 100)
        e+=1
    else:
        s+=1

In [ ]:
print(e/(s+e)*100)

In [ ]:
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1
dist, paths_tmp, global_all_path_dict = get_distance_new(nodes_to_edges_dict, edges_to_counts_dict, line_graph, 'astro-ph', 'math-ph', global_all_path_dict, return_path=True)
dist

In [ ]:
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1

# start_id = 120
# final_id = 120
file_path = pathlib.Path(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle')

global_all_path_dict = {}
    
if True:
    for id_from in trange(start_id, final_id + 1, position=1):
        time_start = time.time()
    
        node_from = node_names[id_from]
        dists = []
        paths = []
        print('id:', id_from)
        for node in tqdm(nodes_to_edges_dict.keys(), position=2):
            if not node == node_from:
                dist, paths_tmp, global_all_path_dict = get_distance_new(nodes_to_edges_dict, edges_to_counts_dict, line_graph, node_from, node, global_all_path_dict, return_path=True)
                dists.append(dist)
                paths.append(paths_tmp)
            else:
                dists.append(0)
                paths.append([])
        dists_dict[node_from] = dists.copy()
        paths_dict[node_from] = paths.copy()
        with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle', 'wb') as handle: # we save calculated distances and corresponding shortest paths to fasten furter analysis
                pickle.dump((dists_dict, paths_dict), handle)
        time_term = time.time()
        print(f'Elapsed time {time_term - time_start}')
    
else:
    with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle', 'rb') as handle:
        dists_dict, paths_dict = pickle.load(handle)

In [ ]:
from tqdm.notebook import tqdm, trange
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1
# start_id = 120
# final_id = 120
file_path = pathlib.Path(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle')

global_all_path_dict = {}

if not file_path.exists():
    time_start = time.time()
    for id_from in trange(start_id, final_id + 1, position=1):
        node_from = node_names[id_from]
        dists = []
        paths = []
        print(id_from, node_from)
        for node in tqdm(nodes_to_edges_dict.keys(), position=2):
            if not node == node_from:
                dist = get_distance_new(nodes_to_edges_dict, edges_to_counts_dict,
                                                                         li, node_from, node)
                dists.append(dist)
            else:
                dists.append(0)
                paths.append([])
        dists_dict[node_from] = dists.copy()
        paths_dict[node_from] = paths.copy()
        with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle',
                  'wb') as handle:  # we save calculated distances and corresponding shortest paths to fasten furter analysis
            pickle.dump((dists_dict, paths_dict), handle)
    time_term = time.time()
    print(f'Elapsed time {time_term - time_start}')
else:
    with open(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle', 'rb') as handle:
        dists_dict, paths_dict = pickle.load(handle)

In [ ]:
# data frame containing calculated distances between every nodes pair
hyper_dist_df = pd.DataFrame(dists_dict)
hyper_dist_df.index = list(nodes_to_edges_dict.keys())
hyper_dist_df

### Weighted clique projection

In [ ]:
# function generating weighted clique projection
# weights = 1 / frequency of appearance
def make_clique_projection_data(edges_to_nodes_dict, edges_to_counts_dict):
    file_path = pathlib.Path(CLIQUE_PATH)

    if not file_path.exists():
        proj_edges_dict = {}
        for edge, nodes in edges_to_nodes_dict.items():
            for i in range(len(nodes) - 1):
                for j in range(i + 1, len(nodes)):
                    node_from = nodes[i]
                    node_to = nodes[j]
                    if not (((node_from, node_to) in proj_edges_dict.keys())
                            or ((node_to, node_from) in proj_edges_dict.keys())):
                        proj_edges_dict[(node_from, node_to)] = edges_to_counts_dict[edge]
                    else:
                        if ((node_from, node_to) in proj_edges_dict.keys()):
                            proj_edges_dict[(node_from, node_to)] += edges_to_counts_dict[edge]
                        else:
                            proj_edges_dict[(node_to, node_from)] += edges_to_counts_dict[edge]

        with open(CLIQUE_PATH, 'w') as handle:
            for (node_from, node_to), weight in proj_edges_dict.items():
                handle.write(f'{node_from} {node_to} {weight}')
                handle.write('\n')

In [ ]:
# first, create cluque projection if it wasn't created previously
make_clique_projection_data(edges_to_nodes_dict, edges_to_counts_dict)

# load and clique projection
proj_graph_file = CLIQUE_PATH
with open(proj_graph_file) as file:
    lines = file.readlines()
    edges_list = []
    weights_list = []
    nodes_dict = {}
    i = -1
    for line in lines:
        from_node, to_node, weight = line.split(" ")

        # from_node = int(from_node)
        # to_node = int(to_node)

        if not from_node in nodes_dict.keys():
            i += 1
            from_id = i
            nodes_dict[from_node] = i
        else:
            from_id = nodes_dict[from_node]
        if not to_node in nodes_dict.keys():
            i += 1
            to_id = i
            nodes_dict[to_node] = i
        else:
            to_id = nodes_dict[to_node]
        edges_list.append([from_id, to_id])
        weights_list.append(1 / float(weight))

proj_graph = ig.Graph(edges_list)
proj_graph.es["weight"] = weights_list

proj_dist_dict = {}
proj_dist_path = {}

# calculation of distances and paths in cluque projection

for from_node in dists_dict.keys():
    from_node_id = nodes_dict[from_node]
    dists = []
    paths_tmp = []
    for node in list(nodes_to_edges_dict.keys()):
        node_id = nodes_dict[node]
        paths = proj_graph.get_shortest_paths(from_node_id, node_id, weights=proj_graph.es["weight"], output="epath")
        paths_tmp.append(paths[0])
        distance = 0
        for e in paths[0]:
            distance += proj_graph.es[e]["weight"]
        dists.append(distance)

    proj_dist_dict[from_node] = dists
    proj_dist_path[from_node] = paths_tmp

# data frame containing dists in clique proj for every nodes pair
proj_dist_df = pd.DataFrame(proj_dist_dict)
proj_dist_df.index = list(nodes_to_edges_dict.keys())
proj_dist_df

### Analysis and comparison

In [ ]:
hyper_dist_df.to_csv(f"results_hypergraph_dist_{data_type}.csv")
proj_dist_df.to_csv(f"results_clique-proj_dist_{data_type}.csv")

In [ ]:
all_hg_dists = []
all_proj_dists = []

for i in range(len(node_names) - 1):
    for j in range(i + 1, len(node_names)):
        from_node = node_names[i]
        to_node = node_names[j]
        if not from_node == to_node:
            all_hg_dists.append(dists_dict[from_node][j])
            all_proj_dists.append(proj_dist_dict[from_node][j])
max_q = 10
hg_quantiles = [np.quantile(all_hg_dists, q / max_q) for q in range(1, max_q)]
proj_quantiles = [np.quantile(all_proj_dists, q / max_q) for q in range(1, max_q)]



In [ ]:
hg_quantiles

In [ ]:
proj_quantiles

In [ ]:
compare_dict = {
    "from node id": [],
    "to node id": [],
    "from node": [],
    "to node": [],
    "hypergraph distance": [],
    "projected distance": [],
    "diff": [],
    "hypergraph rank": [],
    "projected rank": [],
    "rank difference": []
}
for i in range(len(node_names) - 1):
    for j in range(i + 1, len(node_names)):
        from_node = node_names[i]
        to_node = node_names[j]
        hg_dist = dists_dict[from_node][j]
        proj_dist = proj_dist_dict[from_node][j]

        if hg_dist <= hg_quantiles[0]:
            compare_dict["hypergraph rank"].append(1)
        elif hg_dist > hg_quantiles[-1]:
            compare_dict["hypergraph rank"].append(max_q)
        else:
            for q in range(1, max_q - 1):
                if hg_dist > hg_quantiles[q - 1] and hg_dist <= hg_quantiles[q]:
                    compare_dict["hypergraph rank"].append(q + 1)

        if proj_dist <= proj_quantiles[0]:
            compare_dict["projected rank"].append(1)
        elif proj_dist > proj_quantiles[-1]:
            compare_dict["projected rank"].append(max_q)
        else:
            for q in range(1, max_q - 1):
                if proj_dist > proj_quantiles[q - 1] and proj_dist <= proj_quantiles[q]:
                    compare_dict["projected rank"].append(q + 1)

        compare_dict["from node id"].append(i)
        compare_dict["to node id"].append(j)
        compare_dict["from node"].append(from_node)
        compare_dict["to node"].append(to_node)
        compare_dict["hypergraph distance"].append(hg_dist)
        compare_dict["projected distance"].append(proj_dist)
        compare_dict["diff"].append(hg_dist - proj_dist)
        compare_dict["rank difference"].append(
            abs(compare_dict["projected rank"][-1] - compare_dict["hypergraph rank"][-1]))

In [ ]:
compare_df = pd.DataFrame(compare_dict)

In [ ]:
compare_df.sort_values(by=["rank difference"], ascending=False, inplace=True)
compare_df.to_csv(f"compare_{data_type}.csv", sep=",")

In [ ]:
compare_df

In [ ]:
quant_matrix = np.zeros(shape=(max_q, max_q))

N_pairs = len(compare_dict["hypergraph rank"])

for i in range(N_pairs):
    h_rank = compare_dict["hypergraph rank"][i]
    g_rank = compare_dict["projected rank"][i]
    quant_matrix[h_rank - 1, g_rank - 1] += 1 / N_pairs


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(quant_matrix, cmap="binary", )
plt.xticks(range(0, max_q), labels=range(1, max_q + 1))
plt.yticks(range(0, max_q), labels=range(1, max_q + 1))
plt.xlabel('hypergraph distance rank')
plt.ylabel('projected distance rank')
plt.colorbar()
plt.title(data_type[:4])
plt.savefig(f'../figures/ranks_dist_{data_type}.pdf')


In [ ]:
from_name = "q-fin.st"
to_name = "q-fin.rm"
from_id = 65
to_id = 67
dists_dict[from_name][to_id]

In [ ]:
proj_dist_dict[from_name][to_id]

In [ ]:
paths_dict[from_name][to_id]

In [ ]:

for edge in paths_dict[from_name][to_id][0]:
    string = ''
    for x in edges_to_nodes_dict[edge]:
        string += x + ' '
    print(f'{string} weight = {edges_to_counts_dict[edge]}')

In [ ]:
proj_dist_path[from_name][to_id]

In [ ]:
for edge in proj_dist_path[from_name][to_id]:
    print(f'weight = {1 / proj_graph.es[edge]["weight"]}')
    print(node_names[proj_graph.es[edge].source])
    print(node_names[proj_graph.es[edge].target])